In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cafe-sales-dirty-data-for-cleaning-training/dirty_cafe_sales.csv


In [2]:
import pandas as pd

df = pd.read_csv("/kaggle/input/cafe-sales-dirty-data-for-cleaning-training/dirty_cafe_sales.csv")
df.head()


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [3]:

df.info()          # Check column types and missing values


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


> 
* You have 10,000 rows and 8 columns.
* Many columns have missing values.
* All columns are stored as text, even numbers and dates.
* The data needs cleaning before analysis.


In [4]:

df.duplicated().sum()  # Count duplicate rows


0


* No duplicate rows if you checked duplicates.

In [5]:
print(df.isnull().sum())


Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64


* Some columns are mostly filled, but Payment Method and Location have many missing values.

In [6]:
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df['Price Per Unit'] = pd.to_numeric(df['Price Per Unit'], errors='coerce')
df['Total Spent'] = pd.to_numeric(df['Total Spent'], errors='coerce')

df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], errors='coerce')


* Converts text columns to proper types.

* Quantity, Price Per Unit, Total Spent → numbers

* Transaction Date → date/time

* Invalid values that can’t be converted become NaN (missing).

In [7]:
df = df.dropna(subset=['Item', 'Quantity', 'Price Per Unit'])
# removes all rows where any of these columns are missing:

In [8]:
df.isnull().sum()


Transaction ID         0
Item                   0
Quantity               0
Price Per Unit         0
Total Spent          452
Payment Method      2235
Location            2852
Transaction Date     393
dtype: int64

In [9]:
df.info()       

<class 'pandas.core.frame.DataFrame'>
Index: 8713 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    8713 non-null   object        
 1   Item              8713 non-null   object        
 2   Quantity          8713 non-null   float64       
 3   Price Per Unit    8713 non-null   float64       
 4   Total Spent       8261 non-null   float64       
 5   Payment Method    6478 non-null   object        
 6   Location          5861 non-null   object        
 7   Transaction Date  8320 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 612.6+ KB


In [10]:
# Item fixes
df['Item'] = df['Item'].replace(['UNKNOWN', 'ERROR', np.nan], 'Unspecified')

# Payment Method fixes
df['Payment Method'] = df['Payment Method'].replace(['UNKNOWN', 'ERROR', np.nan], 'Unspecified')

# Location fixes
df['Location'] = df['Location'].replace(['UNKNOWN','ERROR', np.nan], 'Unspecified')



* First line fills missing totals using quantity times price.
*  Second line converts the date column into proper date format.

In [11]:
#  Final validations

df = df[df['Quantity'] > 0]
df = df[df['Price Per Unit'] > 0]

# Recompute totals for consistency
df['Total Spent'] = df['Quantity'] * df['Price Per Unit']


df.to_csv("cleaned_cafe_sales.csv", index=False)

print("Done")
print(df)

Done
     Transaction ID         Item  Quantity  Price Per Unit  Total Spent  \
0       TXN_1961373       Coffee       2.0             2.0          4.0   
1       TXN_4977031         Cake       4.0             3.0         12.0   
2       TXN_4271903       Cookie       4.0             1.0          4.0   
3       TXN_7034554        Salad       2.0             5.0         10.0   
4       TXN_3160411       Coffee       2.0             2.0          4.0   
...             ...          ...       ...             ...          ...   
9993    TXN_4766549     Smoothie       2.0             4.0          8.0   
9994    TXN_7851634  Unspecified       4.0             4.0         16.0   
9995    TXN_7672686       Coffee       2.0             2.0          4.0   
9997    TXN_5255387       Coffee       4.0             2.0          8.0   
9999    TXN_6170729     Sandwich       3.0             4.0         12.0   

      Payment Method     Location Transaction Date  
0        Credit Card     Takeaway       2

*  First two lines keep only rows where quantity and price are positive.
*  Then totals are recalculated to make sure they are correct.

> Your cafe sales data is now cleaned. All unknown or invalid entries were corrected, missing totals were fixed, dates were standardized, and only valid rows with positive quantity and price remain. The final dataset is consistent, complete, and ready for analysis or reporting.

In [12]:
df.sample(10)   

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
6749,TXN_1488458,Coffee,4.0,2.0,8.0,Unspecified,Unspecified,2023-08-21
7646,TXN_4913329,Coffee,5.0,2.0,10.0,Credit Card,Takeaway,2023-10-30
2778,TXN_7543369,Tea,5.0,1.5,7.5,Digital Wallet,Takeaway,2023-11-18
5144,TXN_4650611,Coffee,1.0,2.0,2.0,Credit Card,Unspecified,2023-10-14
9066,TXN_5611262,Unspecified,2.0,5.0,10.0,Digital Wallet,In-store,2023-05-22
7973,TXN_7102166,Coffee,4.0,2.0,8.0,Unspecified,Unspecified,2023-12-08
418,TXN_1002457,Cookie,5.0,1.0,5.0,Digital Wallet,Takeaway,2023-09-29
6485,TXN_1128577,Sandwich,1.0,4.0,4.0,Digital Wallet,In-store,2023-11-23
4369,TXN_4168771,Coffee,5.0,2.0,10.0,Digital Wallet,Unspecified,2023-08-02
2063,TXN_8277553,Tea,3.0,1.5,4.5,Cash,Unspecified,2023-09-26


In [13]:
df.isnull().sum()


Transaction ID        0
Item                  0
Quantity              0
Price Per Unit        0
Total Spent           0
Payment Method        0
Location              0
Transaction Date    393
dtype: int64

In [14]:
df[df['Transaction Date'].isna()]


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
11,TXN_3051279,Sandwich,2.0,4.0,8.0,Credit Card,Takeaway,NaT
29,TXN_7640952,Cake,4.0,3.0,12.0,Digital Wallet,Takeaway,NaT
33,TXN_7710508,Unspecified,5.0,1.0,5.0,Cash,Unspecified,NaT
77,TXN_2091733,Salad,1.0,5.0,5.0,Unspecified,In-store,NaT
103,TXN_7028009,Cake,4.0,3.0,12.0,Unspecified,Takeaway,NaT
...,...,...,...,...,...,...,...,...
9933,TXN_9460419,Cake,1.0,3.0,3.0,Unspecified,Takeaway,NaT
9937,TXN_8253472,Cake,1.0,3.0,3.0,Unspecified,Unspecified,NaT
9949,TXN_3130865,Juice,3.0,3.0,9.0,Unspecified,In-store,NaT
9983,TXN_9226047,Smoothie,3.0,4.0,12.0,Cash,Unspecified,NaT


In [15]:
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], errors='coerce')
df = df.dropna(subset=['Transaction Date'])


> 
It converts dates, then drops rows with invalid dates.